In [1]:
from utils import *

# An overview in pratice of AWS's Machine Learning based API services

This notebook demonstrates how to use Amazon Rekognition to perform image analysis. For this demo, we've prepared an image that hosted at Amazon S3. This image will be our main reference to perform visual analysis.

## Initialization

In [31]:
import boto3
import json

aws_region = 'us-east-1'
s3_bucket = "fx-public-samples"

sample_scene = "images/jakarta_skyline.jpg"
sample_meme = "images/obama_aws_meme.png"

rekognition = boto3.client('rekognition', region_name=aws_region)
translate = boto3.client('translate', region_name=aws_region)
comprehend = boto3.client('comprehend', region_name=aws_region)
polly = boto3.client('polly')


# Object and scene detection

![alt text](https://s3.amazonaws.com/fx-public-samples/images/jakarta_skyline.jpg "Logo Title Text 1")

In [1]:
def rekognitionLabel(bucket, key, confidence=70):
    response = rekognition.detect_labels(Image={
        'S3Object': {
            'Bucket': bucket,
            'Name': key,
            }
        },
        MinConfidence=confidence
    )
    return response

In [4]:
response = rekognitionLabel(s3_bucket, sample_scene, 95)
print(json.dumps(response["Labels"], indent=4, sort_keys=True))

[
    {
        "Confidence": 98.32048034667969,
        "Name": "Building"
    },
    {
        "Confidence": 98.32048034667969,
        "Name": "City"
    },
    {
        "Confidence": 98.32048034667969,
        "Name": "Town"
    },
    {
        "Confidence": 98.32048034667969,
        "Name": "Urban"
    }
]


# A more complexe exemple

![alt text](https://s3.amazonaws.com/fx-public-samples/images/obama_aws_meme.png "Logo Title Text 1")

## Celebrity Detection

In [6]:
def rekognitionCelebrity(bucket, key):
    response = rekognition.recognize_celebrities(Image={
        'S3Object': {
            'Bucket': bucket,
            'Name': key,
            }
        }
    )
    return response

In [7]:
response = rekognitionCelebrity(s3_bucket, sample_meme)

celebrities = [k["Name"] for k in response["CelebrityFaces"]]
print("Response: " + json.dumps(celebrities, indent=4, sort_keys=True))

Response: [
    "Madelyn Dunham"
]


## Sentiment Analysis

In [24]:
def rekognitionFace(bucket, key):
    response = rekognition.detect_faces(Image={
        'S3Object': {
            'Bucket': bucket,
            'Name': key,
            }
        }
    )
    return response

In [25]:
response = rekognitionFace(s3_bucket, sample_meme)

#celebrities = [k["Name"] for k in response["FaceDetails"]]
print("Response: " + json.dumps(response, indent=4, sort_keys=True))

Response: {
    "FaceDetails": [
        {
            "BoundingBox": {
                "Height": 0.76106196641922,
                "Left": 0.26875001192092896,
                "Top": 0.10840708017349243,
                "Width": 0.4300000071525574
            },
            "Confidence": 99.88834381103516,
            "Landmarks": [
                {
                    "Type": "eyeLeft",
                    "X": 0.42411622405052185,
                    "Y": 0.3746492266654968
                },
                {
                    "Type": "eyeRight",
                    "X": 0.5723496675491333,
                    "Y": 0.39993369579315186
                },
                {
                    "Type": "nose",
                    "X": 0.48362576961517334,
                    "Y": 0.5046116709709167
                },
                {
                    "Type": "mouthLeft",
                    "X": 0.4044145941734314,
                    "Y": 0.6943173408508301
                },
 

## Text in image

In [26]:
def rekognitionText(bucket, key):
    response = rekognition.detect_text(Image={
        'S3Object': {
            'Bucket': bucket,
            'Name': key,
            }
        }
    )
    return response

In [27]:
response = rekognitionText(s3_bucket, sample_meme)

lines = [k["DetectedText"] for k in response["TextDetections"] if k['Type'] == 'LINE']
print("Number of lines: {}".format(len(lines)))

text = " ".join(lines)
print("Response: " + json.dumps(text, indent=4, sort_keys=True))

Number of lines: 3
Response: "I'M PROUD OF YOU THAT MOMENT WHEN YOU DEPLOY YOUR VERY FIRST AWS LAMBDA FUNCTION USING GO"


# Working with text

Let's use the text extracted above to run some analysis

## Language Detection

In [10]:
def languageDetection(text):
    language = comprehend.detect_dominant_language(
        Text=text
    )['Languages'][0]['LanguageCode']
    return language

In [11]:
language = languageDetection(text)
print("Language: " + language)

Language: en


## Sentiment Detection

In [12]:
def sentimentDetection(text):
    sentiment = comprehend.detect_sentiment(
        Text=text,
        LanguageCode=language
    )
    return sentiment

In [13]:
sentiment = sentimentDetection(text)
print("Sentiment: " + json.dumps(sentiment["Sentiment"], indent=4, sort_keys=True))

Sentiment: "POSITIVE"


## Key Phrases Detection

In [14]:
def keyPhrasesDetection(text):
    response = comprehend.detect_key_phrases(
        Text=text,
        LanguageCode=language
    )
    key_phrases = [k["Text"] for k in response["KeyPhrases"] ]
    return key_phrases

In [15]:
key_phrases = keyPhrasesDetection(text)
print("Key phrases: " + json.dumps(key_phrases, indent=4, sort_keys=True))

Key phrases: [
    "PROUD",
    "THAT MOMENT",
    "YOUR VERY FIRST AWS LAMBDA FUNCTION"
]


## Translation

In [30]:
result = translate.translate_text(Text=text, SourceLanguageCode="en", TargetLanguageCode="de")
print(result['TranslatedText'])

ICH BIN STOLZ AUF SIE, DASS MOMENT, WENN SIE IHRE ALLERERSTE AWS LAMBDA-FUNKTION MIT GO


## Text-to-speech

In [33]:
response = polly.synthesize_speech(
    OutputFormat='mp3',
    Text=text,
    VoiceId='Matthew'
)

# Find me on GitHub

QRCode here